In [1]:
import numpy as np
import pandas as pd
import re
from sklearn import preprocessing

In [40]:
Synthetic_data = pd.read_excel("Synthetic_data.xlsx")

# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

del Synthetic_data['Чтобы ты еще хотел узнать на форуме? Напиши нам об этом']
# del Synthetic_data['Кого бы из спикеров ты бы хотел видеть на форуме?']

# Переименование столбцов в разделе «демография»
Synthetic_data.rename(columns = {'Напиши, сколько тебе полных лет:' : 'Возраст', 'Отметь свой пол' : 'Пол', 'Выбери своё учебное заведение:' : 'ОУ', 'Выбери свой факультет:' : 'Факультет_ГЭУ', "Выбери свой курс:": "Курс", "Выбери профиль своего образования:": "Профиль", "Выбери свой курс:.1": "Курс_не_ГЭУ", "Отметь место проживания до поступления в нынешнее учебное заведение": "МП", "Работаешь ли ты или проходишь стажировку?": "Раб_статус"}, inplace = True)

# очистка пустых ячеек и перевод всех чисел в integer

def recode_empty_cells(dataframe, list_of_columns):

    for column in list_of_columns:
      dataframe[column] = dataframe[column].replace(r'\s+', np.nan, regex=True)
      dataframe[column] = dataframe[column].fillna(0)

    return dataframe

recode_empty_cells(Synthetic_data, ['Возраст', 'Курс'])
Synthetic_data[['Возраст', 'Курс']] = Synthetic_data[['Возраст', 'Курс']].astype(int)

In [41]:
# Разделение большой таблицы исходных данных на пять:

Prostranstvo = Synthetic_data.iloc[:, 2:19]
del Prostranstvo['Какой профиль профессионального обучения ты бы выбрал, если подавал документы сегодня?']

Dosug = Synthetic_data.iloc[:, 49:58]
Demography = Synthetic_data.iloc[:, 2:11]
Dosug = Demography.join(Dosug)

Themes = Synthetic_data.iloc[:, 25:40]
Themes = Demography.join(Themes)

Format = Synthetic_data.iloc[:, 19:25]
Format = Demography.join(Format)

Motivi = Synthetic_data.iloc[:, 40:49]
Motivi = Demography.join(Motivi)

# Переименование столбцов в подтаблицах

Themes.rename(columns = {'Что поможет построить карьеру? Как собрать портфолио в университете? Первое трудоустройство, кейс-чемпионаты и стажировки. / Оцени от 1 до 10' : 'Старт карьеры', 'Инвестиции для начинающих. Как начать получать пассивный доход? / Оцени от 1 до 10' : 'Инвестиции beginner', 'Инвестиции на продвинутом уровне. Что такое фьючерсы и опционы, криптовалютная биржа? / Оцени от 1 до 10' : 'Инвестиции intermediate', 'Все про стартапы: как открыть свой бизнес и получить финансирование? / Оцени от 1 до 10' : 'Стартапы', "Креативные индустрии - как превратить хобби в доход? / Оцени от 1 до 10": "Креативные индустрии", "Импортозамещение в рекламе. Что делать с SMM ввиду блокировки популярных платформ? Новые рекламные инструменты. / Оцени от 1 до 10": "Импортозамещение в технологиях", "Санкции = возможности? Какие навыки будут востребованы в ближайшей перспективе? / Оцени от 1 до 10": "Санкции и возможности", 'Продвижение сферы IT - стоит ли переходить туда? Какие навыки из IT нужны в любой сфере? / Оцени от 1 до 10': 'IT', 'Soft skills - что поможет зарабатывать больше: профессиональные навыки или лидерские качества? / Оцени от 1 до 10': 'Soft skills', "Профессии будущего - каким будет рынок труда через 5 лет? / Оцени от 1 до 10": 'Профессии будущего', 'Как справляться со стрессом, тревогой, быть продуктивнее? Психология и саморазвитие / Оцени от 1 до 10': 'Психология и саморазвитие', 'Тратить меньше или зарабатывать больше? Финансовая грамотность для каждого / Оцени от 1 до 10': 'Фин.грамотность', 'Социальное предпринимательство. Как строить бизнес, помогая обществу? / Оцени от 1 до 10': 'Социальное предпринимательство', 'Мировая экономика. Как адаптироваться бизнесу к новым зарубежным рынкам? / Оцени от 1 до 10': 'Мировая экономика', 'Большие города или малая родина? Где проще строить карьеру? / Оцени от 1 до 10': 'География карьеры'}, inplace = True)

Dosug.rename(columns = {'В перерывах между выступлениями ты бы хотел (шкалу вы знаете: 1 - кто это придумал?, 10 - дайте два билетика, пожалуйста): / Послушать выступление Stand Up комика' : 'Stand Up', 'В перерывах между выступлениями ты бы хотел (шкалу вы знаете: 1 - кто это придумал?, 10 - дайте два билетика, пожалуйста): / Послушать выступление музыкальной группы' : 'Муз. группа', 'В перерывах между выступлениями ты бы хотел (шкалу вы знаете: 1 - кто это придумал?, 10 - дайте два билетика, пожалуйста): / Посетить фотозону' : 'Фотозона', 'В перерывах между выступлениями ты бы хотел (шкалу вы знаете: 1 - кто это придумал?, 10 - дайте два билетика, пожалуйста): / Посетить информационные стойки компаний с их представителями' : 'Инф.стойки', "В перерывах между выступлениями ты бы хотел (шкалу вы знаете: 1 - кто это придумал?, 10 - дайте два билетика, пожалуйста): / Перекусить или выпить кофе": "Перекус", "В перерывах между выступлениями ты бы хотел (шкалу вы знаете: 1 - кто это придумал?, 10 - дайте два билетика, пожалуйста): / Посидеть в тишине": "Тишина", "В перерывах между выступлениями ты бы хотел (шкалу вы знаете: 1 - кто это придумал?, 10 - дайте два билетика, пожалуйста): / Поиграть в настольные игры и с кем-то познакомиться": "Знакомства"}, inplace = True)

Prostranstvo.rename(columns = {'Мне комфортнее, когда:' : 'Питание', 'Мне комфортнее, когда:.1' : 'Активность в перерыве', 'Мне комфортнее, когда:.2' : 'Время мероприятий', 'Мне комфортнее, когда:.3' : 'Навигация', "Мне комфортнее, когда:.4": "Доступ на мероприятия", "Мне комфортнее, когда:.5": "Дресс-код", "Мне комфортнее, когда:.6": "Информационное обеспечение"}, inplace = True)

Format.rename(columns = {'Диалог на равных (X откровенно отвечает на ваши вопросы) / Оцени от 1 до 10' : 'Диалог на равных', 'Мастер-класс (X учит вас тому, в чем разбирается) / Оцени от 1 до 10' : 'Мастер-класс', 'Дискуссия (Вы совместно с X и еще несколькими спикерами обсуждаете актуальную тему, делая выводы) / Оцени от 1 до 10' : 'Дискуссия', 'Форсайт (Вместе с вами X создает картину будущего) / Оцени от 1 до 10' : 'Форсайт', "Выступление (X рассказывает о своих взглядах на проблему и ее решение) / Оцени от 1 до 10": "Выступление", "Кейс (Вы в командах работаете с кейсами, подготовленными X, защищаете и анализируете их) / Оцени от 1 до 10": "Кейс"}, inplace = True)

Motivi.rename(columns = {'Что тебя замотивирует посетить форум? / Новые знакомства' : 'Знакомства', 'Что тебя замотивирует посетить форум? / Поиск работодателя' : 'Поиск работы', 'Что тебя замотивирует посетить форум? / Определение карьерной траектории' : 'Карьерная траектория', 'Что тебя замотивирует посетить форум? / Новые знания и навыки' : 'Новые ЗУНы', "Что тебя замотивирует посетить форум? / Участие в масштабном мероприятии": "Участие в масштабном мероприятии", "Что тебя замотивирует посетить форум? / Спикер, который мне нравится": "Спикер", "Что тебя замотивирует посетить форум? / Понимание текущей экономической ситуации": "Понимание экономической ситуации", 'Что тебя замотивирует посетить форум? / Отдых с друзьями': 'Отдых с друзьями', 'Что тебя замотивирует посетить форум? / Альтернатива занятиям': 'Альтернатива занятиям'}, inplace = True)

# очистка пустых ячеек и перевод всех чисел в integer

recode_empty_cells(Dosug, ['Stand Up', 'Муз. группа', 'Фотозона', 'Инф.стойки', 'Перекус', 'Тишина', 'Знакомства'])
Dosug[['Stand Up', 'Муз. группа', 'Фотозона', 'Инф.стойки', 'Перекус', 'Тишина', 'Знакомства']] = Dosug[['Stand Up', 'Муз. группа', 'Фотозона', 'Инф.стойки', 'Перекус', 'Тишина', 'Знакомства']].astype(int)

recode_empty_cells(Themes, ['Старт карьеры', 'Инвестиции beginner', 'Инвестиции intermediate', 'Стартапы', 'Креативные индустрии', 'Импортозамещение в технологиях', 'Санкции и возможности', 'IT', 'Soft skills', 'Профессии будущего', 'Психология и саморазвитие', 'Фин.грамотность', 'Социальное предпринимательство', 'Мировая экономика', 'География карьеры'])
Themes[['Старт карьеры', 'Инвестиции beginner', 'Инвестиции intermediate', 'Стартапы', 'Креативные индустрии', 'Импортозамещение в технологиях', 'Санкции и возможности', 'IT', 'Soft skills', 'Профессии будущего', 'Психология и саморазвитие', 'Фин.грамотность', 'Социальное предпринимательство', 'Мировая экономика', 'География карьеры']] = Themes[['Старт карьеры', 'Инвестиции beginner', 'Инвестиции intermediate', 'Стартапы', 'Креативные индустрии', 'Импортозамещение в технологиях', 'Санкции и возможности', 'IT', 'Soft skills', 'Профессии будущего', 'Психология и саморазвитие', 'Фин.грамотность', 'Социальное предпринимательство', 'Мировая экономика', 'География карьеры']].astype(int)

recode_empty_cells(Motivi, ['Знакомства', 'Поиск работы', 'Карьерная траектория', 'Новые ЗУНы', 'Участие в масштабном мероприятии', 'Спикер', 'Понимание экономической ситуации', 'Отдых с друзьями', 'Альтернатива занятиям'])
Motivi[['Знакомства', 'Поиск работы', 'Карьерная траектория', 'Новые ЗУНы', 'Участие в масштабном мероприятии', 'Спикер', 'Понимание экономической ситуации', 'Отдых с друзьями', 'Альтернатива занятиям']] = Motivi[['Знакомства', 'Поиск работы', 'Карьерная траектория', 'Новые ЗУНы', 'Участие в масштабном мероприятии', 'Спикер', 'Понимание экономической ситуации', 'Отдых с друзьями', 'Альтернатива занятиям']].astype(int)

# среднее и СКО

Dosug['СКО'] = Dosug.iloc[:, 9:16].std(axis=1)
Dosug = Dosug.set_index('СКО') # переносим СКО в столбец индексов

Themes['СКО'] = Themes.iloc[:, 9:24].std(axis=1)
Themes = Themes.set_index('СКО')

Format['СКО'] = Format.iloc[:, 9:15].std(axis=1)
Format = Format.set_index('СКО')

Motivi['СКО'] = Motivi.iloc[:, 9:18].std(axis=1)
Motivi = Motivi.set_index('СКО')

# возвращаем СКО в таблицу и добавляем среднее значение по строкам

Dosug.reset_index(drop=True, inplace=True)
Dosug['СКО'] = Dosug.iloc[:, 9:16].std(axis=1)
Dosug['среднее'] = Dosug.iloc[:, 9:16].mean(axis=1)

Themes.reset_index(drop=True, inplace=True)
Themes['СКО'] = Themes.iloc[:, 9:16].std(axis=1)
Themes['среднее'] = Themes.iloc[:, 9:16].mean(axis=1)

Format.reset_index(drop=True, inplace=True)
Format['СКО'] = Format.iloc[:, 9:16].std(axis=1)
Format['среднее'] = Format.iloc[:, 9:16].mean(axis=1)

Motivi.reset_index(drop=True, inplace=True)
Motivi['СКО'] = Motivi.iloc[:, 9:16].std(axis=1)
Motivi['среднее'] = Motivi.iloc[:, 9:16].mean(axis=1)

# нормировка

normalized_Dosug = preprocessing.normalize(Dosug.iloc[:, 9:16])
normalized_Dosug = pd.DataFrame(normalized_Dosug, columns=['Stand Up норм.', 'Муз. группа норм.', 'Фотозона норм.', 'Инф.стойки норм.', 'Перекус норм.', 'Тишина норм.', 'Знакомства норм.'])
Dosug = Dosug.join(normalized_Dosug)

normalized_Themes = preprocessing.normalize(Themes.iloc[:, 9:24])
normalized_Themes = pd.DataFrame(normalized_Themes, columns=['Старт карьеры норм.', 'Инвестиции beginner норм.', 'Инвестиции intermediate норм.', 'Стартапы норм.', 'Креативные индустрии норм.', 'Импортозамещение в технологиях норм.', 'Санкции и возможности норм.', 'IT норм.', 'Soft skills норм.', 'Профессии будущего норм.', 'Психология и саморазвитие норм.', 'Фин.грамотность норм.', 'Социальное предпринимательство норм.', 'Мировая экономика норм.', 'География карьеры норм.'])
Themes = Themes.join(normalized_Themes)

normalized_Format = preprocessing.normalize(Format.iloc[:, 9:15])
normalized_Format = pd.DataFrame(normalized_Format, columns=['Диалог на равных норм.', 'Мастер-класс группа норм.', 'Дискуссия норм.', 'Форсайт норм.', 'Выступление норм.', 'Кейс норм.'])
Format = Format.join(normalized_Format)

normalized_Motivi = preprocessing.normalize(Motivi.iloc[:, 9:18])
normalized_Motivi = pd.DataFrame(normalized_Motivi, columns=['Знакомства  норм.', 'Поиск работы норм.', 'Карьерная траектория норм.', 'Новые ЗУНы норм.', 'Участие в масштабном мероприятии норм.', 'Спикер норм.', 'Понимание экономической ситуации норм.', 'Отдых с друзьями норм.', 'Альтернатива занятиям норм.'])
Motivi = Motivi.join(normalized_Motivi)

In [42]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('S_Data.xlsx')

# Write each dataframe to a different worksheet.
Prostranstvo.to_excel(writer, sheet_name='Prostranstvo')
Dosug.to_excel(writer, sheet_name='Dosug')
Themes.to_excel(writer, sheet_name='Themes')
Format.to_excel(writer, sheet_name='Format')
Motivi.to_excel(writer, sheet_name='Motivi')

# Close the Pandas Excel writer and output the Excel file.
writer.save()